In [4]:
import os

# Check if the notebook is running on Colab
if 'COLAB_GPU' in os.environ:
    # This block will run only in Google Colab
    IN_COLAB = True
    print("Running on Google Colab. Cloning the repository.")
    !git clone https://github.com/pedro15sousa/energy-based-models-compression.git
    %cd energy-based-models-compression/notebooks
else: 
    # This block will run if not in Google Colab
    IN_COLAB = False
    print("Not running on Google Colab. Assuming local environment.")

Not running on Google Colab. Assuming local environment.


In [5]:
import sys
sys.path.append('..')  # This adds the parent directory (main_folder) to the Python path

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import MNIST
import torch.utils.data as data
import torch.nn.utils.prune as prune

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning
    import pytorch_lightning as pl
# Callbacks
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
# Pytorch Summary
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install --quiet torchsummary
    from torchsummary import summary

import numpy as np
import pandas as pd
import copy

## Imports for plotting
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from matplotlib import cm
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

from metrics.classifier import VGG
from EBM import DeepEnergyModel
from callbacks import InceptionScoreCallback, \
    FIDCallback, SamplerCallback, OutlierCallback, \
    GenerateImagesCallback

import shutil
if IN_COLAB:
    from google.colab import files, drive
    drive.mount('/content/drive')

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models"
DRIVE_PATH = "/content/drive/My Drive/EBM_saved_models/"

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device: ", device)

Device:  cpu


/var/folders/pl/gjvtfzbj4691nr_0pszjdspw0000gn/T/ipykernel_81710/3206500048.py:35: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [7]:
# Transformations applied on each image => make them a tensor and normalize between -1 and 1
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                               ])

# Loading the training dataset. We need to split it into a training and validation part
train_set = MNIST(root=DATASET_PATH, train=True, transform=transform, download=True)

# Loading the test set
test_set = MNIST(root=DATASET_PATH, train=False, transform=transform, download=True)

# We define a set of data loaders that we can use for various purposes later.
# Note that for actually training a model, we will use different data loaders
# with a lower batch size.
train_loader = data.DataLoader(train_set, batch_size=64, shuffle=True,  drop_last=True,  num_workers=2, pin_memory=True)
test_loader  = data.DataLoader(test_set,  batch_size=128, shuffle=False, drop_last=False, num_workers=2)

In [8]:
if os.path.exists('../saved_models/mnist-classifier-1 (1).pth'):
    # Load the best model
    mnist_classifier = VGG()

    if device == 'cuda':
        mnist_classifier.load_state_dict(torch.load('../saved_models/mnist-classifier-1 (1).pth'))
    else:
        mnist_classifier.load_state_dict(torch.load('../saved_models/mnist-classifier-1 (1).pth', map_location=torch.device('cpu')))

    mnist_classifier.to(device)
    print("Model already exists and loaded.")
    summary(mnist_classifier, input_size=(1, 28, 28))
else:
    print("Classifier not found in saved_models. Please run the classifier notebook first.")

Model already exists and loaded.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 28, 28]           1,280
              ReLU-2          [-1, 128, 28, 28]               0
            Conv2d-3          [-1, 128, 28, 28]         147,584
              ReLU-4          [-1, 128, 28, 28]               0
            Conv2d-5          [-1, 128, 28, 28]         147,584
              ReLU-6          [-1, 128, 28, 28]               0
         MaxPool2d-7          [-1, 128, 14, 14]               0
            Conv2d-8          [-1, 256, 14, 14]         295,168
              ReLU-9          [-1, 256, 14, 14]               0
           Conv2d-10          [-1, 256, 14, 14]         590,080
             ReLU-11          [-1, 256, 14, 14]               0
           Conv2d-12          [-1, 256, 14, 14]         590,080
             ReLU-13          [-1, 256, 14, 14]               0
      

In [9]:
def apply_unstructured_pruning(model, backbone=True, amount=0.5):
    """Applies unstructured L1 pruning to the specified layers in the model.
    
    Args:
        model (nn.Module): The neural network model to prune.
        backbone (bool): If True, prune only Conv2d layers. If False, prune all layers.
        amount (float): The fraction of weights to prune.
    """
    for module in model.modules():
        # If backbone is True, prune only Conv2d layers
        if backbone and isinstance(module, nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
        # If backbone is False, prune all layers with weights
        elif not backbone and hasattr(module, 'weight'):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')

In [10]:
def apply_structured_pruning(model, backbone=True, amount=0.5):
    """Applies structured Ln pruning to the specified layers in the model.
    
    Args:
        model (nn.Module): The neural network model to prune.
        backbone (bool): If True, prune only Conv2d layers. If False, prune all layers.
        amount (float): The fraction of weights to prune.
    """
    for module in model.modules():
        # If backbone is True, prune only Conv2d layers
        if backbone and isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)  # dim=0 for filter pruning)
            prune.remove(module, 'weight')
        # If backbone is False, prune all layers with weights
        elif not backbone and hasattr(module, 'weight'):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)  # dim=0 for filter pruning)
            prune.remove(module, 'weight')

In [11]:
def get_pruned_model(model, struct=False, backbone=True, amount=0.2):
    pruned_model = copy.deepcopy(model)
    if struct:
        apply_structured_pruning(model=pruned_model, backbone=backbone, amount=amount)
    else:
        apply_unstructured_pruning(model=pruned_model, backbone=backbone, amount=amount)
    
    return pruned_model


In [12]:
pretrained_filename = os.path.join(CHECKPOINT_PATH, "MNIST_resnet18.ckpt")
model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
# model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
summary(model, input_size=(1, 28, 28))
# pl.seed_everything(43)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           3,200
             Swish-2           [-1, 64, 14, 14]               0
         MaxPool2d-3             [-1, 64, 7, 7]               0
            Conv2d-4             [-1, 64, 7, 7]          36,928
             Swish-5             [-1, 64, 7, 7]               0
            Conv2d-6             [-1, 64, 7, 7]          36,928
             Swish-7             [-1, 64, 7, 7]               0
BasicResidualBlock-8             [-1, 64, 7, 7]               0
            Conv2d-9             [-1, 64, 7, 7]          36,928
            Swish-10             [-1, 64, 7, 7]               0
           Conv2d-11             [-1, 64, 7, 7]          36,928
            Swish-12             [-1, 64, 7, 7]               0
BasicResidualBlock-13             [-1, 64, 7, 7]               0
           Conv2d-14            [-1, 1

In [16]:
def train_pruned_model(stru, ratio, pruned_model):
    perc = str(ratio * 100)
    default_root_dir = os.path.join(CHECKPOINT_PATH, f"MNIST/{perc}/{stru}")
    save_path = default_root_dir if IN_COLAB else DRIVE_PATH
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=default_root_dir,
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=7,
                         gradient_clip_val=0.1,
                         callbacks=[ModelCheckpoint(dirpath=save_path, filename='MNIST_pruned_{ratio}_{stru}-{epoch:02d}', save_top_k=-1, every_n_epochs=1),
                                    GenerateImagesCallback(every_n_epochs=3),
                                    SamplerCallback(every_n_epochs=3),
                                    OutlierCallback(),
                                    LearningRateMonitor("epoch"),
                                    InceptionScoreCallback(mnist_classifier),
                                    FIDCallback(mnist_classifier)
                                   ])

    pretrained_filename = os.path.join(CHECKPOINT_PATH, f'MNIST_resnet_pruned_{ratio}_{stru}.ckpt') if IN_COLAB else os.path.join(DRIVE_PATH, f'MNIST_resnet_pruned_{ratio}_{stru}.ckpt')
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
    else:
        print("No pretrained model found. Start training from scratch...")
        pl.seed_everything(42)
        model = pruned_model
    
    # Print pruning statistics
    model.print_pruning_stats()

    trainer.fit(model, train_loader, test_loader)

    model = DeepEnergyModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
        
    # No testing as we are more interested in other properties
    return model

## Unstructured

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir ../saved_models/MNIST/10.0/unstructured/lightning_logs

In [ ]:
un_10_model = get_pruned_model(model, struct=False, backbone=False, amount=0.1)

un_10_model = train_pruned_model("unstructured",
                    ratio=0.5,
                    pruned_model=un_10_model)

In [ ]:
un_20_model = get_pruned_model(model, struct=False, backbone=False, amount=0.2)

un_20_model = train_pruned_model("unstructured",
                    ratio=0.5,
                    pruned_model=un_20_model)

In [ ]:
un_30_model = get_pruned_model(model, struct=False, backbone=False, amount=0.3)

un_30_model = train_pruned_model("unstructured",
                    ratio=0.5,
                    pruned_model=un_10_model)